Let us first import the libraries we will need for today's workshop. 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


We will also need these dependencies so make sure you install them

In [ ]:
pip install openpyxl

Now let's open the data file as a dataframe. Note that so far we have worked with `.csv` files but Python can just as easily open and manipulate other file types like `excel`. Have a quick look at your data 

In [ ]:
df = pd.read_excel("covid_dataset.xlsx")
df.head(10)

Notice that not all columns are shown, as Python has a cap on how many columns it can show in a table. To help us visually the data better, we will remove that cap.

In [ ]:
pd.set_option('display.max_columns', None)
df.head()

Now you can see all 79 columns. Please go through each column and try to understand what it is that it's measuring.

There is a lot of information contained in this dataset. But for today's tutorial we do not need all of this information. Let's try to curate a smaller dataset which only contains the variables that we are interested in. We are going to keep several demographic variables, some information about COVID19 severity and 3 the results of 3 cognitive tasks.

In [ ]:
df2 = pd.concat([df.loc[:, "userID":"First Language binary"], df["Mechanical ventilation (1-Y,0-no)"], df.loc[:, "rs_spatialSpan":"rs_verbalAnalogies"]], axis=1)

In [ ]:
df2.head()

Our dataset looks a lot more manageable now! However, some of the variable names we have are not very intuitive. Let's change the name of the cognitive tasks so that it is easier to understand what they measure.

In [ ]:
df2.rename(columns = {'rs_spatialSpan':'memory_spatialSpan', 'rs_verbalAnalogies':'language_verbalAnalogies', 'rs_targetDetection':'attention_targetDetection'}, inplace = True)
df2.head()

Try to change the column called `Qualifications` to `Education`

In [ ]:
#CODE HERE

Now, let's look at the data types for each of our variables. We discussed these during the primer, but as a quick summary, it is important that the data type matches what we expect. Strings are usually saved as objects in python and numeric variables are saved as integers (int64) or float (float64). However, sometimes either due to humar error or due to the way the data is saved, Python might assign the wrong data type to your variables (e.g. numbers may be saved as strings). When this happens, statistical tests throw errors and so do certain plotting functions and it can take a while to figure out where the errors. Have a look at our data and see if you can spot any wrong data types.

In [ ]:
df2.dtypes

It looks like Age was saved as an object, instead of integer. Let's change that. First, we get rid of the quotation marks around the numbers, then we tell python to convert those numbers to `int`

In [ ]:
df2.Age = df["Age"].str.strip('\"') 


In [ ]:
df2.Age = df2.Age.astype(int)

In [ ]:
df2.dtypes # check that the changes have worked


Now that we have all of our data in the right format, we may want to check if the values contained in the columns are consistent. If you have a look in the `First language binary` column you will notice that some of the values are capitalised while others are not. This is a problem as any statistics we run will assume that `english` and `English` are two distinct groups. To fix this, we can choose a consistent way to show this and stick with it.

In [ ]:
df2['First Language binary'] = df2['First Language binary'].str.title()
df2.head()

Try to do the same for the `Handedness` variable.

In [ ]:
#CODE HERE

We may also wish to binarise some of our variables. This is usually done as specific statistical techniques require the data to be in a numeric format. So, it is always safer to have your data saved in a numeric format, wherever possible. For instance, the language variable is a good example where binarization would be useful. Let's see how this can be done below.

In [ ]:
# Text Data of First Language is converted into Numeric Binary Data
df_one = pd.get_dummies(df2['First Language binary'])
print(df_one)

Note that the two variables that were outputted are each other's opposites, so we only need one of them. Let us choose the first one, though it does not matter which one you choose as long as you understand what the values represent.

In [ ]:
df2["First Language binary"] = df_one.English #Replace the old values with the numeric ones
df2.rename(columns = {'First Language binary':'First Language 1-English 0-Other'}, inplace = True) #change the name of the column to something more meaningful
df2.head()

Try to do apply the same concept of binarisation to the `Handedness` variable

In [ ]:
#CODE HERE

Now, before moving forward we may also want to check the validity of the data. Are there any odd values that are not within the expected range? The best way to examine this is via plotting. Let's visualise some of the variables in the dataset

In [ ]:
plt.hist(df2.memory_spatialSpan)

In [ ]:
plt.hist(df2.attention_targetDetection)

In [ ]:
plt.hist(df2.language_verbalAnalogies)

In [ ]:
plt.hist(df2.Age)

We see that for the cognitive variables the data looks valid. However, when we look at age, one person seems to be over 600 years old! Unless we are dealing with a vampire, we may want to do something about this data. There are multiple ways to deal with this type of data but the simplest and most straightforward is to get rid of it, by replacing it with a NaN value.

In [ ]:
print(df2.Age[df2.Age > 100]) #show all the people who had an age over 100
df2.Age[df2.Age > 100] = np.NaN  #replace with NaNs anyone who is older than 100
plt.hist(df2.Age) #plot the new data


Check for the remaining variables if there is any data that is not valid. If you find anything, replace it with NaNs

Now that we have established that the data looks healthy, the next thing on the list is to check whether there are any duplicated values in our dataset. The easiest way to check this is via the number of unique user IDs. If we had no duplicates, then the number of unique IDs should correspond with the total number of rows in our data.

In [ ]:
print(len(np.unique(df2.userID)))
print(len(df2.userID))

You can also use the `duplicated` function to obtain the same results.

In [ ]:
sum(df2.duplicated()) 

It looks like we got two duplicates in our data. Let us try to remove them.

In [ ]:

df2 = df2[~df2.duplicated()] #Have a look at lecture 8 if you don't remember this function
sum(df2.duplicated()) # calculate the number of duplicates again after removing them. If we were succcessful none should be there anymore.

We are almost done cleaning our data. Now let's check if there are any missing values. It's generally good practice to get rid of participants with missing data or replace those values with plausible estimates (like the mean/median). In this case, we will show the first. To find out if we have any missing values we can use the `isna` function. This returns True when there is a missing value and False when there isn't one.

In [ ]:
df2.isna()

With big datasets it can be quite hard to visualise where the missing values are. So let's instead calculate the number of missing values that each person has.

In [ ]:
sum_missing = df2.isna().sum(axis=1) #This shows us how many missing values each person has
print(sum_missing)  
print(sum(sum_missing > 0)) #how many people have at least one missing data


Let's get rid of all the people who have at least one missing value

In [ ]:
df2 = df2[~(sum_missing > 0)]

Another important check we need to perform is to look for outliers. These are often less easy to spot and sometimes it is unclear if one value is an outlier or not. Some may remove more extreme values only because it helps with their hypothesis, creating bias. Just because a value is extreme it does not always mean that it is wrong. However, if you don't do something about your outliers, they will likely bias your results and your data.

One robust, objective way to deal with outliers is via a process called winsorisation. This is replaces the extreme values in our data with slightly less extreme values, in order to limit the effect of the outliers on the calculations or the results. The mean value calculated after such replacement of the extreme values is called winsorized mean. 

For example, 90% winsorization means the replacement of the top 5% and bottom 5% of the data. The top 5% of the data is replaced by the value of the data at the 95th percentile and the value of the bottom 5% of the data is replaced by the value of the data at the 5th percentile. 

Let's see if our cognitive data requires any winsorisation by visualising it.

In [ ]:
plt.boxplot(df2.language_verbalAnalogies)

In [ ]:
plt.boxplot(df2.memory_spatialSpan)

In [ ]:
plt.boxplot(df2.attention_targetDetection)

Looking at the box plots, we can see that for the memory task, we have one person who performed really well. A score of 8 is plausible within this task, but not very common. For that reason, it is not valid for us to just remove it. Instead, let us apply the winsorisation procedure to replace the value with something a bit less extreme by replacing the values above the 95th percentile with the value of the score on the 95th percentile.

In [ ]:
from scipy.stats.mstats import winsorize


winsorized_memory = winsorize(df2['memory_spatialSpan'],(0.05,0.05))  #apply winsorisation
  
plt.boxplot(winsorized_memory)  #visualise the data after winsorisation
plt.title('Winsorized array')
plt.show()

It now looks a lot better! Let's replace the old values in the memory task with the winsorised version.

In [ ]:
df2['memory_spatialSpan'] = winsorized_memory

One final check, now that the data is in the right format and we are confident that it is correct, is to ensure everything is on the same scale. Different tasks will have different scoring mechanisms, on some tasks the scores may range from 4 to 8, while in others they may range between 30 and 70. This is often a problem as depending on the statistical test you use, if one variable has very large values, it will dominate over other variables. So the features with higher variability may receive more weight than the others even though in theory they should not.

If we look at our cognitive data, we can see that the tests are on different scales. To fix this, we can standardise our data so that each task has a mean of 0 and a standard deviation of 1. 

In [ ]:
memory_task = (df2['memory_spatialSpan'] - df2['memory_spatialSpan'].mean())/df2['memory_spatialSpan'].std()

In [ ]:
language_task = (df2['language_verbalAnalogies'] - df2['language_verbalAnalogies'].mean())/df2['language_verbalAnalogies'].std()

In [ ]:
attention_task = (df2['attention_targetDetection'] - df2['attention_targetDetection'].mean())/df2['attention_targetDetection'].std()

In the above code we subtracted the mean values and divided by the standard deviation, to obtain standardised values. Then, we old values with the standardised ones. We can visualise these changes to better understand how our data looks now.

In [ ]:
plt.boxplot(language_task)

In [ ]:
plt.boxplot(attention_task)

In [ ]:
plt.boxplot(memory_task)

However, `standardisation` assumes that our data is normally distributed. If we look at the distributions of some of our cognitive tests we can see that some are not exactly normal. A different approach that does not assume a normal distribution is called `normalisation`. The formula for this is similar to the standardisation, but you use minimum and maximum values instead of mean and standard deviation

`X_normalised = (X - X_min) / (X_max - X_min)`

The output that you get is a scale where all the values range from 0 to 1 (and sometimes from -1 to 1). This approach is particularly useful when there are no outliers in your data. If the data has a lot of outliers that you cannot remove, then this method will not work very well, and it is recommended that you use `standardisation` instead. Since we made sure to get ride of outliers in our data, let's proceed with the `normalisation` technique.

In [ ]:
df2['memory_spatialSpan'] = (df2['memory_spatialSpan'] - df2['memory_spatialSpan'].min())/(df2['memory_spatialSpan'].max() - df2['memory_spatialSpan'].min()) 

In [ ]:
df2['language_verbalAnalogies'] = (df2['language_verbalAnalogies'] - df2['language_verbalAnalogies'].min())/(df2['language_verbalAnalogies'].max() - df2['language_verbalAnalogies'].min()) 

In [ ]:
df2['attention_targetDetection'] = (df2['attention_targetDetection'] - df2['attention_targetDetection'].min())/(df2['attention_targetDetection'].max() - df2['attention_targetDetection'].min()) 

Great, we are finally done with the data cleaning!

Let's run some analyses with the what we obtained so far.